# Модель генерации долгосрочной памяти

## Формируем датасет для обучения модели

In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Загрузка модели с параметром attn_implementation="eager"
model = GPT2LMHeadModel.from_pretrained("gpt2", output_attentions=True, attn_implementation="eager")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Установка pad_token как eos_token
tokenizer.pad_token = tokenizer.eos_token
model.eval()


# Функция для выделения значимых частей контекста
def extract_significant_context(context, current_fragment):
    # Конкатенация context и current_fragment
    combined_input = context + " " + current_fragment
    inputs = tokenizer(combined_input, return_tensors="pt")

    # Получение выходов модели с учетом внимания
    outputs = model(**inputs)

    # Извлечение attention weights из последнего слоя
    attentions = outputs.attentions[-1].detach().numpy()

    # Токенизация контекста и текущего фрагмента
    context_tokens = tokenizer.tokenize(context)
    current_fragment_tokens = tokenizer.tokenize(current_fragment)

    # Определение границы контекста и текущего фрагмента
    context_length = len(context_tokens)

    # Суммирование внимания по всем головам и слоям
    combined_attention = attentions.mean(axis=1).mean(axis=1)

    # Средние значения внимания для токенов контекста
    context_attention_scores = combined_attention[:, :context_length].mean(axis=0)

    # Выбор значимых токенов с высоким вниманием
    significant_indices = context_attention_scores.argsort()[-5:][::-1]  # Выбираем топ-5 по значимости
    significant_tokens = [context_tokens[i] for i in significant_indices]

    # Возвращаем значимые токены
    return significant_tokens


# Пример данных
context = 'Famitsu enjoyed the story , and were particularly pleased with the improvements to gameplay . Japanese gaming site Game Watch Impress , despite negatively noting its pacing and elements recycled from previous games , was generally positive about its story and characters , and found its gameplay entertaining despite off @-@ putting difficulty spikes . 4Gamer.net writer Naohiko Misuosame , in a " Play Test " article based on the game \'s PSN demo , felt that Valkyria Chronicles III provided a " pro'
current_fragment = "PlayStation Official Magazine - UK praised the story 's blurring of Gallia 's moral standing , art style , and most points about its gameplay , positively noting the latter for both its continued quality and the tweaks to balance and content . Its one major criticism were multiple difficulty spikes , something that had affected the previous games . Heath Hindman of gaming website PlayStation Lifestyle praised the addition of non @-@ linear elements and improvements or removal of mechanics from V"

# Получение значимых частей контекста
significant_tokens = extract_significant_context(context, current_fragment)

# Вывод значимых токенов
print("Significant context tokens:", significant_tokens)

Significant context tokens: ['F', 'Ġ.', 'Ġ,', 'Ġenjoyed', 'ĠJapanese']


In [17]:
import json
import torch
import time
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import plotly.graph_objects as go

# Загрузка токенайзера и модели GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", output_attentions=True, attn_implementation="eager")
model.eval()

# Установка pad_token как eos_token
tokenizer.pad_token = tokenizer.eos_token


# Функция загрузки указанного датасета
def load_intermediate_dataset(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return data


# Функция для вычисления ошибки предсказания
def compute_loss(model, input_text, target_text):
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    target_ids = tokenizer(target_text, return_tensors="pt", padding=True, truncation=True)["input_ids"]

    # Синхронизация длин входных данных и целевых меток
    input_len = inputs["input_ids"].size(1)
    target_len = target_ids.size(1)

    if input_len != target_len:
        min_len = min(input_len, target_len)
        inputs["input_ids"] = inputs["input_ids"][:, :min_len]
        target_ids = target_ids[:, :min_len]
        inputs["attention_mask"] = inputs["attention_mask"][:, :min_len]  # Обрезаем attention_mask также

    outputs = model(**inputs, labels=target_ids)
    return outputs.loss.item()


# Функция для выделения значимых частей контекста
def extract_influential_context_tokens(context, current_fragment):
    combined_input = context + " " + current_fragment
    inputs = tokenizer(combined_input, return_tensors="pt")
    outputs = model(**inputs)

    attentions = outputs.attentions[-1].mean(dim=1).squeeze(0)

    context_tokens = tokenizer.tokenize(context)
    context_length = len(context_tokens)

    influence_scores = attentions[:, :context_length].sum(dim=0)

    significant_indices = influence_scores.argsort(descending=True)[:5]
    significant_tokens = [context_tokens[i] for i in significant_indices]

    significant_context = tokenizer.convert_tokens_to_string(significant_tokens)
    return significant_context


# Загрузка данных
intermediate_dataset_path = "./intermediate_dataset.json"
data = load_intermediate_dataset(intermediate_dataset_path)

# Список для хранения результатов
results = []

# Оценка времени обработки первых 5 записей
start_time = time.time()
for i, entry in enumerate(data[:5]):
    context = entry["context"]
    current_fragment = entry["current_fragment"]
    target = entry["target"]

    # Пересчет потерь для полной версии контекста (raw_loss)
    combined_input = context + " " + current_fragment
    raw_loss = compute_loss(model, combined_input, target)

    # Пересчет потерь для текущего фрагмента без контекста (fragment_loss)
    fragment_loss = compute_loss(model, current_fragment, target)

    # Извлечение значимых токенов контекста и пересчет потерь
    significant_context = extract_influential_context_tokens(context, current_fragment)
    combined_significant_input = significant_context + " " + current_fragment
    significant_loss = compute_loss(model, combined_significant_input, target)

    results.append({"index": i, "raw_loss": raw_loss, "fragment_loss": fragment_loss, "significant_loss": significant_loss, "difference": fragment_loss - raw_loss})

end_time = time.time()

# Расчет среднего времени обработки одной записи и оценка времени для всего датасета
average_time_per_record = (end_time - start_time) / 5
estimated_total_time = average_time_per_record * len(data)

print(f"Среднее время обработки одной записи: {average_time_per_record:.4f} секунд")
print(f"Оценка общего времени обработки датасета: {estimated_total_time:.2f} секунд")

# Продолжение обработки всего датасета
for i, entry in enumerate(data[5:], start=5):
    context = entry["context"]
    current_fragment = entry["current_fragment"]
    target = entry["target"]

    # Пересчет потерь для полной версии контекста (raw_loss)
    combined_input = context + " " + current_fragment
    raw_loss = compute_loss(model, combined_input, target)

    # Пересчет потерь для текущего фрагмента без контекста (fragment_loss)
    fragment_loss = compute_loss(model, current_fragment, target)

    # Извлечение значимых токенов контекста и пересчет потерь
    significant_context = extract_influential_context_tokens(context, current_fragment)
    combined_significant_input = significant_context + " " + current_fragment
    significant_loss = compute_loss(model, combined_significant_input, target)

    results.append({"index": i, "raw_loss": raw_loss, "fragment_loss": fragment_loss, "significant_loss": significant_loss, "difference": fragment_loss - raw_loss})

# Создание DataFrame и сортировка по разнице fragment_loss - raw_loss
df = pd.DataFrame(results)
df_sorted = df.sort_values(by="difference")

# Построение графика
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_sorted["index"], y=df_sorted["raw_loss"], mode="lines+markers", name="Raw Loss"))

fig.add_trace(go.Scatter(x=df_sorted["index"], y=df_sorted["fragment_loss"], mode="lines+markers", name="Fragment Loss"))

fig.add_trace(go.Scatter(x=df_sorted["index"], y=df_sorted["significant_loss"], mode="lines+markers", name="Significant Context Loss"))

fig.update_layout(title="Loss Values vs. Record Index (Sorted by Fragment Loss - Raw Loss)", xaxis_title="Record Index", yaxis_title="Loss Value", legend_title="Loss Type")

fig.show()

Среднее время обработки одной записи: 0.4993 секунд
Оценка общего времени обработки датасета: 98.37 секунд


In [18]:
import json
import torch
import time
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import plotly.graph_objects as go

# Загрузка токенайзера и модели GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", output_attentions=True, attn_implementation="eager")
model.eval()

# Установка pad_token как eos_token
tokenizer.pad_token = tokenizer.eos_token


# Функция загрузки указанного датасета
def load_intermediate_dataset(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return data


# Функция для вычисления ошибки предсказания
def compute_loss(model, input_text, target_text):
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    target_ids = tokenizer(target_text, return_tensors="pt", padding=True, truncation=True)["input_ids"]

    # Синхронизация длин входных данных и целевых меток
    input_len = inputs["input_ids"].size(1)
    target_len = target_ids.size(1)

    if input_len != target_len:
        min_len = min(input_len, target_len)
        inputs["input_ids"] = inputs["input_ids"][:, :min_len]
        target_ids = target_ids[:, :min_len]
        inputs["attention_mask"] = inputs["attention_mask"][:, :min_len]  # Обрезаем attention_mask также

    outputs = model(**inputs, labels=target_ids)
    return outputs.loss.item()


# Функция для выделения значимых частей контекста
def extract_influential_context_tokens(context, current_fragment):
    combined_input = context + " " + current_fragment
    inputs = tokenizer(combined_input, return_tensors="pt")
    outputs = model(**inputs)

    attentions = outputs.attentions[-1].mean(dim=1).squeeze(0)

    context_tokens = tokenizer.tokenize(context)
    context_length = len(context_tokens)

    influence_scores = attentions[:, :context_length].sum(dim=0)

    significant_indices = influence_scores.argsort(descending=True)[: len(context if context is not None else "") / 3]
    significant_tokens = [context_tokens[i] for i in significant_indices]

    significant_context = tokenizer.convert_tokens_to_string(significant_tokens)
    return significant_context


# Загрузка данных
intermediate_dataset_path = "./intermediate_dataset.json"
data = load_intermediate_dataset(intermediate_dataset_path)

# Список для хранения результатов
results = []

# Оценка времени обработки первых 5 записей
start_time = time.time()
for i, entry in enumerate(data[:5]):
    context = entry["context"]
    current_fragment = entry["current_fragment"]
    target = entry["target"]
    raw_loss = entry["raw_loss"]
    fragment_loss = entry["fragment_loss"]

    significant_context = extract_influential_context_tokens(context, current_fragment)

    combined_significant_input = significant_context + " " + current_fragment
    print("\n!!!!!!!!!!!!!!!!!! " + context + "\n================ " + significant_context + "\n---------------- " + current_fragment)
    significant_loss = compute_loss(model, combined_significant_input, target)

    results.append({"index": i, "raw_loss": raw_loss, "fragment_loss": fragment_loss, "significant_loss": significant_loss, "difference": fragment_loss - raw_loss})

end_time = time.time()

# Расчет среднего времени обработки одной записи и оценка времени для всего датасета
average_time_per_record = (end_time - start_time) / 5
estimated_total_time = average_time_per_record * len(data)

print(f"Среднее время обработки одной записи: {average_time_per_record:.4f} секунд")
print(f"Оценка общего времени обработки датасета: {estimated_total_time:.2f} секунд")

# Продолжение обработки всего датасета
for i, entry in enumerate(data[5:], start=5):
    context = entry["context"]
    current_fragment = entry["current_fragment"]
    target = entry["target"]
    raw_loss = entry["raw_loss"]
    fragment_loss = entry["fragment_loss"]

    significant_context = extract_influential_context_tokens(context, current_fragment)

    combined_significant_input = significant_context + " " + current_fragment
    significant_loss = compute_loss(model, combined_significant_input, target)

    results.append({"index": i, "raw_loss": raw_loss, "fragment_loss": fragment_loss, "significant_loss": significant_loss, "difference": fragment_loss - raw_loss})

# Создание DataFrame и сортировка по разнице fragment_loss - raw_loss
df = pd.DataFrame(results)
df_sorted = df.sort_values(by="difference")

# Построение графика
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_sorted["index"], y=df_sorted["raw_loss"], mode="lines+markers", name="Raw Loss"))

fig.add_trace(go.Scatter(x=df_sorted["index"], y=df_sorted["fragment_loss"], mode="lines+markers", name="Fragment Loss"))

fig.add_trace(go.Scatter(x=df_sorted["index"], y=df_sorted["significant_loss"], mode="lines+markers", name="Significant Context Loss"))

fig.update_layout(title="Loss Values vs. Record Index (Sorted by Fragment Loss - Raw Loss)", xaxis_title="Record Index", yaxis_title="Loss Value", legend_title="Loss Type")

fig.show()

c:\git\MUIV\concl\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884



KeyboardInterrupt: 

In [20]:
import json
import torch
import time
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import plotly.graph_objects as go

# Загрузка токенайзера и модели GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", output_attentions=True, attn_implementation="eager")
model.eval()

# Установка pad_token как eos_token
tokenizer.pad_token = tokenizer.eos_token


# Функция загрузки указанного датасета
def load_intermediate_dataset(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return data


# Функция для вычисления ошибки предсказания
def compute_loss(model, input_text, target_text):
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    target_ids = tokenizer(target_text, return_tensors="pt", padding=True, truncation=True)["input_ids"]

    # Синхронизация длин входных данных и целевых меток
    input_len = inputs["input_ids"].size(1)
    target_len = target_ids.size(1)

    if input_len != target_len:
        min_len = min(input_len, target_len)
        inputs["input_ids"] = inputs["input_ids"][:, :min_len]
        target_ids = target_ids[:, :min_len]
        inputs["attention_mask"] = inputs["attention_mask"][:, :min_len]  # Обрезаем attention_mask также

    outputs = model(**inputs, labels=target_ids)
    return outputs.loss.item()


# Функция для выделения значимых частей контекста
def extract_influential_context_tokens(context, current_fragment):
    combined_input = context + " " + current_fragment
    inputs = tokenizer(combined_input, return_tensors="pt")
    outputs = model(**inputs)

    attentions = outputs.attentions[-1].mean(dim=1).squeeze(0)

    context_tokens = tokenizer.tokenize(context)
    context_length = len(context_tokens)

    influence_scores = attentions[:, :context_length].sum(dim=0)

    significant_indices = influence_scores.argsort(descending=True)[:5]
    significant_tokens = [context_tokens[i] for i in significant_indices]

    significant_context = tokenizer.convert_tokens_to_string(significant_tokens)
    return significant_context


# Загрузка данных
intermediate_dataset_path = "./intermediate_dataset.json"
data = load_intermediate_dataset(intermediate_dataset_path)

# Список для хранения результатов
results = []

# Оценка времени обработки первых 5 записей
start_time = time.time()
for i, entry in enumerate(data[:5]):
    context = entry["context"]
    current_fragment = entry["current_fragment"]
    target = entry["target"]

    # Пересчет потерь для полной версии контекста (raw_loss)
    combined_input = context + " " + current_fragment
    raw_loss = compute_loss(model, combined_input, target)

    # Пересчет потерь для текущего фрагмента без контекста (fragment_loss)
    fragment_loss = compute_loss(model, current_fragment, target)

    # Извлечение значимых токенов контекста и пересчет потерь
    significant_context = extract_influential_context_tokens(context, current_fragment)
    combined_significant_input = significant_context + " " + current_fragment
    significant_loss = compute_loss(model, combined_significant_input, target)

    results.append({"index": i, "raw_loss": raw_loss, "fragment_loss": fragment_loss, "significant_loss": significant_loss, "difference": fragment_loss - raw_loss})

end_time = time.time()

# Расчет среднего времени обработки одной записи и оценка времени для всего датасета
average_time_per_record = (end_time - start_time) / 5
estimated_total_time = average_time_per_record * len(data)

print(f"Среднее время обработки одной записи: {average_time_per_record:.4f} секунд")
print(f"Оценка общего времени обработки датасета: {estimated_total_time:.2f} секунд")

# Продолжение обработки всего датасета
for i, entry in enumerate(data[5:], start=5):
    context = entry["context"]
    current_fragment = entry["current_fragment"]
    target = entry["target"]

    # Пересчет потерь для полной версии контекста (raw_loss)
    combined_input = context + " " + current_fragment
    raw_loss = compute_loss(model, combined_input, target)

    # Пересчет потерь для текущего фрагмента без контекста (fragment_loss)
    fragment_loss = compute_loss(model, current_fragment, target)

    # Извлечение значимых токенов контекста и пересчет потерь
    significant_context = extract_influential_context_tokens(context, current_fragment)
    combined_significant_input = significant_context + " " + current_fragment
    significant_loss = compute_loss(model, combined_significant_input, target)

    results.append({"index": i, "raw_loss": raw_loss, "fragment_loss": fragment_loss, "significant_loss": significant_loss, "difference": fragment_loss - raw_loss})

# Создание DataFrame и сортировка по разнице fragment_loss - raw_loss
df = pd.DataFrame(results)
df_sorted = df.sort_values(by="difference")

# Построение графика
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_sorted["index"], y=df_sorted["raw_loss"], mode="markers", name="Raw Loss"))

fig.add_trace(go.Scatter(x=df_sorted["index"], y=df_sorted["fragment_loss"], mode="markers", name="Fragment Loss"))

fig.add_trace(go.Scatter(x=df_sorted["index"], y=df_sorted["significant_loss"], mode="markers", name="Significant Context Loss"))

fig.update_layout(title="Loss Values vs. Record Index (Sorted by Fragment Loss - Raw Loss)", xaxis_title="Record Index", yaxis_title="Loss Value", legend_title="Loss Type")

fig.show()

c:\git\MUIV\concl\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884



Среднее время обработки одной записи: 0.5306 секунд
Оценка общего времени обработки датасета: 104.52 секунд


In [21]:
import json
import torch
import time
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import plotly.graph_objects as go

# Загрузка токенайзера и модели GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", output_attentions=True, attn_implementation="eager")
model.eval()

# Установка pad_token как eos_token
tokenizer.pad_token = tokenizer.eos_token


# Функция загрузки указанного датасета
def load_intermediate_dataset(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return data


# Функция для вычисления ошибки предсказания
def compute_loss(model, input_text, target_text):
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    target_ids = tokenizer(target_text, return_tensors="pt", padding=True, truncation=True)["input_ids"]

    # Синхронизация длин входных данных и целевых меток
    input_len = inputs["input_ids"].size(1)
    target_len = target_ids.size(1)

    if input_len != target_len:
        min_len = min(input_len, target_len)
        inputs["input_ids"] = inputs["input_ids"][:, :min_len]
        target_ids = target_ids[:, :min_len]
        inputs["attention_mask"] = inputs["attention_mask"][:, :min_len]  # Обрезаем attention_mask также

    outputs = model(**inputs, labels=target_ids)
    return outputs.loss.item()


# Функция для выделения значимых частей контекста
def extract_influential_context_tokens(context, current_fragment):
    combined_input = context + " " + current_fragment
    inputs = tokenizer(combined_input, return_tensors="pt")
    outputs = model(**inputs)

    attentions = outputs.attentions[-1].mean(dim=1).squeeze(0)

    context_tokens = tokenizer.tokenize(context)
    context_length = len(context_tokens)

    influence_scores = attentions[:, :context_length].sum(dim=0)

    significant_indices = influence_scores.argsort(descending=True)[:5]
    significant_tokens = [context_tokens[i] for i in significant_indices]

    significant_context = tokenizer.convert_tokens_to_string(significant_tokens)
    return significant_context


# Загрузка данных
intermediate_dataset_path = "./intermediate_dataset.json"
data = load_intermediate_dataset(intermediate_dataset_path)

# Список для хранения результатов
results = []

# Оценка времени обработки первых 5 записей
start_time = time.time()
for i, entry in enumerate(data[:5]):
    context = entry["context"]
    current_fragment = entry["current_fragment"]
    target = entry["target"]

    # Пересчет потерь для полной версии контекста (raw_loss)
    combined_input = context + " " + current_fragment
    raw_loss = compute_loss(model, combined_input, target)

    # Пересчет потерь для текущего фрагмента без контекста (fragment_loss)
    fragment_loss = compute_loss(model, current_fragment, target)

    # Извлечение значимых токенов контекста и пересчет потерь
    significant_context = extract_influential_context_tokens(context, current_fragment)
    combined_significant_input = significant_context + " " + current_fragment
    significant_loss = compute_loss(model, combined_significant_input, target)

    # Сохраняем только если fragment_loss >= raw_loss
    if fragment_loss >= raw_loss:
        results.append({"index": i, "raw_loss": raw_loss, "fragment_loss": fragment_loss, "significant_loss": significant_loss, "difference": fragment_loss - raw_loss})

end_time = time.time()

# Расчет среднего времени обработки одной записи и оценка времени для всего датасета
average_time_per_record = (end_time - start_time) / 5
estimated_total_time = average_time_per_record * len(data)

print(f"Среднее время обработки одной записи: {average_time_per_record:.4f} секунд")
print(f"Оценка общего времени обработки датасета: {estimated_total_time:.2f} секунд")

# Продолжение обработки всего датасета
for i, entry in enumerate(data[5:], start=5):
    context = entry["context"]
    current_fragment = entry["current_fragment"]
    target = entry["target"]

    # Пересчет потерь для полной версии контекста (raw_loss)
    combined_input = context + " " + current_fragment
    raw_loss = compute_loss(model, combined_input, target)

    # Пересчет потерь для текущего фрагмента без контекста (fragment_loss)
    fragment_loss = compute_loss(model, current_fragment, target)

    # Извлечение значимых токенов контекста и пересчет потерь
    significant_context = extract_influential_context_tokens(context, current_fragment)
    combined_significant_input = significant_context + " " + current_fragment
    significant_loss = compute_loss(model, combined_significant_input, target)

    # Сохраняем только если fragment_loss >= raw_loss
    if fragment_loss >= raw_loss:
        results.append({"index": i, "raw_loss": raw_loss, "fragment_loss": fragment_loss, "significant_loss": significant_loss, "difference": fragment_loss - raw_loss})

# Создание DataFrame и сортировка по разнице fragment_loss - raw_loss
df = pd.DataFrame(results)
df_sorted = df.sort_values(by="difference")

# Построение графика
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_sorted["index"], y=df_sorted["raw_loss"], mode="lines+markers", name="Raw Loss"))

fig.add_trace(go.Scatter(x=df_sorted["index"], y=df_sorted["fragment_loss"], mode="lines+markers", name="Fragment Loss"))

fig.add_trace(go.Scatter(x=df_sorted["index"], y=df_sorted["significant_loss"], mode="lines+markers", name="Significant Context Loss"))

fig.update_layout(title="Loss Values vs. Record Index (Sorted by Fragment Loss - Raw Loss)", xaxis_title="Record Index", yaxis_title="Loss Value", legend_title="Loss Type")

fig.show()

c:\git\MUIV\concl\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884



Среднее время обработки одной записи: 0.5690 секунд
Оценка общего времени обработки датасета: 112.10 секунд


In [22]:
df_sorted = df.sort_values(by="difference")

# Построение графика
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_sorted["index"], y=df_sorted["raw_loss"], mode="markers", name="Raw Loss"))

fig.add_trace(go.Scatter(x=df_sorted["index"], y=df_sorted["fragment_loss"], mode="markers", name="Fragment Loss"))

fig.add_trace(go.Scatter(x=df_sorted["index"], y=df_sorted["significant_loss"], mode="markers", name="Significant Context Loss"))

fig.update_layout(title="Loss Values vs. Record Index (Sorted by Fragment Loss - Raw Loss)", xaxis_title="Record Index", yaxis_title="Loss Value", legend_title="Loss Type")

fig.show()

# Обучаем модель сжимать контекст

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Загрузка предобученной модели и токенайзера GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


# Подготовка данных для обучения
def tokenize_function(examples):
    # Токенизация контекста и текущего фрагмента, соединенных вместе
    return tokenizer(examples["context"] + " " + examples["current_fragment"], truncation=True)


# Загрузка и обработка данных
from datasets import load_dataset

# Загрузка вашего кастомного датасета
dataset = load_dataset("json", data_files="./intermediate_dataset.json")

# Токенизация данных
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Используем data collator с маскированием
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)  # Использование masked language modeling  # Вероятность маскирования токенов

# Параметры обучения
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Создание тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
)

# Обучение модели
trainer.train()


# Функция для выделения значимых токенов
def extract_significant_tokens(model, tokenizer, context, current_fragment):
    inputs = tokenizer(context + " " + current_fragment, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=50, do_sample=False)
    significant_tokens = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return significant_tokens


# Пример использования
context = "Ваш текст контекста здесь..."
current_fragment = "Ваш текущий фрагмент здесь..."
significant_tokens = extract_significant_tokens(model, tokenizer, context, current_fragment)
print("Significant tokens to store:", significant_tokens)